In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

from keras.layers import BatchNormalization
from keras.layers import Dropout

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Bayer/PacakgingLineDataSet.csv', encoding = 'unicode_escape', nrows= 200000, low_memory=False)

In [ ]:
print(df.shape)
df.head()

(200000, 670)


,Date_Time,ActivityName,Category,ProductionYN,GRZ_L09_02_Filler_APP_VarPersistent_VC_MD_955_Z_R_CP_BadTubesDisch_V,GRZ_L09_07_Case_Packer_CrtClos_FlapfTR_cl_V,GRZ_L09_04_Cartoner_SX_V0102_00_ProductInsertedCompl_VC_MD_102_Z_A_CD_ProdNotInserted_V,GRZ_L09_04_Cartoner_SX_V0102_00_ProductInsertedCompl_VC_MX_102_MF_ProdNotInsert_V,GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tGlCoolCirSW_tCoolCir_tCoolCir_3__fFlowRate_l_min_V,GRZ_L09_07_Case_Packer_CrtClos_FlapfTL_cl_V,...,GRZ_L09_06__Bundler_Application_SpF_HMI_SpF_M_F0_A_FilmEnd_14067_V,GRZ_L09_07_Case_Packer_STâ__RUNG_S17_V,GRZ_L09_07_Case_Packer_STâ__RUNG_S345_V,GRZ_L09_07_Case_Packer_STâ__RUNG_S336_V,GRZ_L09_07_Case_Packer_STâ__RUNG_S347_V,GRZ_L09_08_Palletizer_IDB_IrbPal_St5Prod_V,GRZ_L09_08_Palletizer_Eject_full_V,GRZ_L09_08_Palletizer_IDB_IrbPal_St5Ok_V,GRZ_L09_08_Palletizer_Scan_no_code_V,Generated Date
0,2020-06-08 20:13:58,Undefiniert,2 Unplanned downtime,0,967,0,2934,0,2.29,0,...,0,0,0,0,0,0,1,1,0,2020-06-08 20:13:58
1,2020-06-08 20:13:59,Undefiniert,2 Unplanned downtime,0,967,0,2934,0,2.17,0,...,0,0,0,0,0,0,1,1,0,2020-06-08 20:13:59
2,2020-06-08 20:14:00,Undefiniert,2 Unplanned downtime,0,967,0,2934,0,2.14,0,...,0,0,0,0,0,0,1,1,0,2020-06-08 20:14:00
3,2020-06-08 20:14:01,Undefiniert,2 Unplanned downtime,0,967,0,2934,0,2.18,0,...,0,0,0,0,0,0,1,1,0,2020-06-08 20:14:01
4,2020-06-08 20:14:02,Undefiniert,2 Unplanned downtime,0,967,0,2934,0,2.12,0,...,0,0,0,0,0,0,1,1,0,2020-06-08 20:14:02


In [ ]:
selFeat = ['GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tGlCoolCirSW_tCoolCir_tCoolCir_3__fFlowRate_l_min_V',
 'GRZ_L09_06__Bundler_Application_SeaF_HMI_SeaF_V_r_PosAct_V',
 'GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tFoCupEdSW_tData_tData_25__fMonDiffMm_V',
 'GRZ_L09_07_Case_Packer_STâ__RUNG_FaultNrFu13_V',
 'GRZ_L09_04_Cartoner_SX_V0803_01_Efficiency_VC_MD_803_Z_A_CT_MachWorkingTime_V',
 'GRZ_L09_07_Case_Packer_STâ__RUNG_FaultNrFu8_V',
 'GRZ_L09_08_Palletizer_IDB_Stat1_Wnd_L_InfStopCyl_AirPrOk_V',
 'GRZ_L09_07_Case_Packer_CrtClos_FlapfTL_op_V',
 'GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tDrvPiPlT1SW_tEject_cLevelBinFaulty_V',
 'GRZ_L09_08_Palletizer_IDB_Stat1_Wnd_L_TurnCylInf_TiAlarm_V',
 'GRZ_L09_07_Case_Packer_STâ__RUNG_FaultNrFu12_V',
 'GRZ_L09_06__Bundler_Application_Mach_HMI_Mach_F_dw_PLC_HMI_ComCheck_V',
 'GRZ_L09_07_Case_Packer_CrtLd_FlapHoMa_op_V',
 'GRZ_L09_08_Palletizer_IDB_Stat1_Wnd_L_InfStopCyl_TiAlarm_V',
 'GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tDrvMainSW_fVelCurBlMin_V',
 'GRZ_L09_07_Case_Packer_CrpBkhTp_V',
 'GRZ_L09_08_Palletizer_READY_RU_V',
 'GRZ_L09_08_Palletizer_IDB_Stat1_Wnd_L_TurnCylInf_AirPrOk_V',
 'GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tDrvMainSW_fVelCurCycMin_V',
 'GRZ_L09_08_Palletizer_IDB_PalClp_TiAlarm_V',
 'GRZ_L09_07_Case_Packer_CrpBkhBt_V',
 'GRZ_L09_04_Cartoner_SX_V3455_00_RobotLift_VC_MD_3455_A_LiftPosAct_V',
 'GRZ_L09_02_Filler_APP_V0003_00_OpUnits_HMI_VC_MD_003_A_MachineState_V',
 'GRZ_L09_07_Case_Packer_CrtLd_FlapHoDi_op_V',
 'GRZ_L09_06__Bundler_Application_SeaB_HMI_SeaB_V_r_PosAct_V',
 'GRZ_L09_07_Case_Packer_STâ__RUNG_FaultNrFu9_V',
 'GRZ_L09_07_Case_Packer_STâ__RUNG_FaultNrFu14_V',
 'GRZ_L09_08_Palletizer_IDB_PalClp_AirPrOk_V',
 'GRZ_L09_06__Bundler_Application_Mach_HMI_Mach_F_dw_HMI_PLC_ComCheck_V',
 'GRZ_L09_07_Case_Packer_CrtClos_FlapfTR_op_V',
 'GRZ_L09_02_Filler_APP_V2200_00_BoxInfeed_VC_MX_2200_TE_EmptyBoxInfeed_V',
 'GRZ_L09_08_Palletizer_STâ__RUNG_FaultNrFu12_V',
 'GRZ_L09_08_Palletizer_AutoActiv_V',
 'GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tBFSenSplSW_tSen_lCntBat_V',
 'GRZ_L09_08_Palletizer_Auto_V',
 'GRZ_L09_08_Palletizer_STâ__RUNG_FaultNrFu1_V',
 'GRZ_L09_07_Case_Packer_CartonSuportDown_V',
 'GRZ_L09_06__Bundler_Application_Ouf1_HMI_Ouf1_V_r_VelAct_V',
 'GRZ_L09_06__Bundler_Application_Ver_HMI_Ver_V_r_VelAct_V',
 'GRZ_L09_04_Cartoner_SX_V0456_01_RobotTool_VC_MX_456_TE_VacMonitor_V',
 'GRZ_L09_04_Cartoner_SX_V3700_99_UpstreamMach_VC_MX_3700_MW_UpstrNotReady_V',
 'GRZ_L09_04_Cartoner_SX_V0100_00_ProductSensingTubes_IX_100_B2ProdSens_V',
 'GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tFdFilCtrlSW_tSen_wSenPre_V',
 'GRZ_L09_07_Case_Packer_VacSensSup_V',
 'GRZ_L09_06__Bundler_Application_Ouf2_HMI_Ouf2_V_r_PosAct_V',
 'GRZ_L09_06__Bundler_Application_Inf_HMI_Inf_V_r_VelAct_V',
 'GRZ_L09_07_Case_Packer_CARTSUP_V',
 'GRZ_L09_06__Bundler_Application_Ouf3_HMI_Ouf3_V_r_VelAct_V',
 'GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tFdFilCtrlSW_tSen_wEje_V',
 'GRZ_L09_06__Bundler_Application_SeaF_HMI_SeaF_V_w_ProdContTimeAct_V',
 'GRZ_L09_04_Cartoner_SX_V0456_01_RobotTool_IX_456_B5VacLow_V',
 'GRZ_L09_06__Bundler_Application_Inf_HMI_Inf_V_w_CycleTimeAct_V',
 'GRZ_L09_04_Cartoner_SX_V0100_00_ProductSensingApplicator_IX_100_B3ProdSens_V',
 'GRZ_L09_06__Bundler_Application_FiSl_HMI_FiSl_V_r_ActTime_op_V',
 'GRZ_L09_07_Case_Packer_TP27_PROGSTRING_TAKT_MIN_V',
 'GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tFdFilCtrlSW_tSen_wEjePre_V',
 'GRZ_L09_06__Bundler_Application_SeaJ_HMI_SeaJ_V_r_ActTime_fr_V',
 'GRZ_L09_02_Filler_APP_V0710_01_CombiCartoner_VC_MS_710_MU_CombiCartoner_iState_V',
 'GRZ_L09_08_Palletizer_VakPump_V',
 'GRZ_L09_08_Palletizer_HMPTAKT_V',
 'GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tFdFilCtrlSW_tSen_wSen_V',
 'GRZ_L09_08_Palletizer_MESSZEIT_PROD_V',
 'GRZ_L09_08_Palletizer_EmStON_V',
 'GRZ_L09_07_Case_Packer_HMPTAKT_V',
 'GRZ_L09_04_Cartoner_SX_V0803_01_Efficiency_VC_MD_803_Z_A_CT_ScrapTime_V',
 'GRZ_L09_06__Bundler_Application_Ouf2_HMI_Ouf2_V_r_VelAct_V',
 'GRZ_L09_08_Palletizer_TP27_PROGSTRING_TAKT_MIN_V',
 'GRZ_L09_07_Case_Packer_MESSZEIT_PROD_V',
 'GRZ_L09_04_Cartoner_SX_V0983_00_EnergySaveMode_VC_MX_983_MW_ESMActivated_V']

In [ ]:
thisDf = df[selFeat]

In [ ]:
thisDf.shape

(200000, 69)

In [ ]:

thisDf['ProductionYN'] = df['ProductionYN']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(thisDf.shape)
thisDf.head()

(200000, 70)


,GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tGlCoolCirSW_tCoolCir_tCoolCir_3__fFlowRate_l_min_V,GRZ_L09_06__Bundler_Application_SeaF_HMI_SeaF_V_r_PosAct_V,GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tFoCupEdSW_tData_tData_25__fMonDiffMm_V,GRZ_L09_07_Case_Packer_STâ__RUNG_FaultNrFu13_V,GRZ_L09_04_Cartoner_SX_V0803_01_Efficiency_VC_MD_803_Z_A_CT_MachWorkingTime_V,GRZ_L09_07_Case_Packer_STâ__RUNG_FaultNrFu8_V,GRZ_L09_08_Palletizer_IDB_Stat1_Wnd_L_InfStopCyl_AirPrOk_V,GRZ_L09_07_Case_Packer_CrtClos_FlapfTL_op_V,GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tDrvPiPlT1SW_tEject_cLevelBinFaulty_V,GRZ_L09_08_Palletizer_IDB_Stat1_Wnd_L_TurnCylInf_TiAlarm_V,...,GRZ_L09_01_Blistering_PLC_Modules__Default__Global_PV_tFdFilCtrlSW_tSen_wSen_V,GRZ_L09_08_Palletizer_MESSZEIT_PROD_V,GRZ_L09_08_Palletizer_EmStON_V,GRZ_L09_07_Case_Packer_HMPTAKT_V,GRZ_L09_04_Cartoner_SX_V0803_01_Efficiency_VC_MD_803_Z_A_CT_ScrapTime_V,GRZ_L09_06__Bundler_Application_Ouf2_HMI_Ouf2_V_r_VelAct_V,GRZ_L09_08_Palletizer_TP27_PROGSTRING_TAKT_MIN_V,GRZ_L09_07_Case_Packer_MESSZEIT_PROD_V,GRZ_L09_04_Cartoner_SX_V0983_00_EnergySaveMode_VC_MX_983_MW_ESMActivated_V,ProductionYN
0,2.29,-29.06,0.16,1,677715646,111,0,0,0,30,...,0,0,1,0,9807774,0.0,0,0,0,0
1,2.17,-29.06,0.16,1,677715646,111,0,0,0,30,...,0,0,1,0,9807774,0.0,0,0,0,0
2,2.14,-29.06,0.16,1,677715646,111,0,0,0,30,...,0,0,1,0,9807774,0.0,0,0,0,0
3,2.18,-29.06,0.16,1,677775648,111,0,0,0,30,...,0,0,1,0,9807774,0.0,0,0,0,0
4,2.12,-29.06,0.16,1,677775648,111,0,0,0,30,...,0,0,1,0,9807774,0.0,0,0,0,0


In [ ]:

from sklearn.model_selection import train_test_split

train, test = train_test_split(thisDf, test_size=0.25, random_state= 42)
print(train.shape)
print(test.shape)


(150000, 70)
(50000, 70)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
train_scaled = pd.DataFrame(train_scaled)
test_scaled = scaler.transform(test)
test_scaled = pd.DataFrame(test_scaled)

print(train_scaled.shape)

print(test_scaled.shape)

(150000, 70)
(50000, 70)


In [ ]:
N = 69
X_train = train_scaled.iloc[:, :N]
y_train=  train_scaled.iloc[:, 69]

X_test = test_scaled.iloc[:, :N]
y_test=  test_scaled.iloc[:, 69]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(150000, 69)
(50000, 69)
(150000,)
(50000,)


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model = Sequential()
model.add(Dense(20, input_dim=69, kernel_initializer='he_uniform', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32,kernel_initializer='he_uniform', activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))


In [ ]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

# fit the model
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=0)

# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print(accuracy)
print(f1_score)

0.9745799899101257
0.9673541784286499
